# Google 日历工具集

> [Google 日历](https://workspace.google.com/intl/en-419/products/calendar/) 是 Google Workspace 的一款产品，可让用户整理日程和活动。这是一个基于云的日历，允许用户创建、编辑和删除活动。它还允许用户与其他用户共享他们的日历。

## 概述

本笔记本将帮助您开始使用 Google 日历工具集。此工具集可与 Google 日历 API 交互，对日历执行各种操作。它允许您：

- 创建活动。
- 搜索活动。
- 更新活动。
- 将活动移至不同的日历。
- 删除活动。
- 列出活动。

## 设置

要使用此工具集，您需要：

1. 拥有一个有权访问 Google 日历的 Google 帐户。
2. 按照[Google 日历 API 文档](https://developers.google.com/calendar/api/quickstart/python#authorize_credentials_for_a_desktop_application)中的说明设置您的凭据。下载 `credentials.json` 文件后，您就可以开始使用 Google 日历 API 了。

要启用对单个工具的自动化跟踪，请设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### 安装

此工具包位于 [langchain-google](https://github.com/langchain-ai/langchain-google) 仓库的 `langchain-google-community` 包中。我们需要 `calendar` 扩展：

In [ ]:
%pip install -qU langchain-google-community\[calendar\]

##实例化

默认情况下，工具包会读取本地的 `credentials.json` 文件。您也可以手动提供一个 `Credentials` 对象。

In [ ]:
from langchain_google_community import CalendarToolkit

toolkit = CalendarToolkit()

### 自定义认证

在后台，`googleapi` 资源是使用以下方法创建的。你可以手动构建一个 `googleapi` 资源以获得更多认证控制。

In [2]:
from langchain_google_community import CalendarToolkit
from langchain_google_community.calendar.utils import (
    build_resource_service,
    get_google_credentials,
)

# Can review scopes here: https://developers.google.com/calendar/api/auth
# For instance, readonly scope is https://www.googleapis.com/auth/calendar.readonly
credentials = get_google_credentials(
    token_file="token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="credentials.json",
)

api_resource = build_resource_service(credentials=credentials)
toolkit = CalendarToolkit(api_resource=api_resource)

## 工具
查看可用工具：

In [5]:
tools = toolkit.get_tools()
tools

[CalendarCreateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x10ad13fb0>),
 CalendarSearchEvents(api_resource=<googleapiclient.discovery.Resource object at 0x10ad13fb0>),
 CalendarUpdateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x10ad13fb0>),
 GetCalendarsInfo(api_resource=<googleapiclient.discovery.Resource object at 0x10ad13fb0>),
 CalendarMoveEvent(api_resource=<googleapiclient.discovery.Resource object at 0x10ad13fb0>),
 CalendarDeleteEvent(api_resource=<googleapiclient.discovery.Resource object at 0x10ad13fb0>),
 GetCurrentDatetime(api_resource=<googleapiclient.discovery.Resource object at 0x10ad13fb0>)]

- [CalendarCreateEvent](https://python.langchain.com/api_reference/google_community/calendar/langchain_google_community.calendar.create_event.CalendarCreateEvent.html)
- [CalendarSearchEvents](https://python.langchain.com/api_reference/google_community/calendar/langchain_google_community.calendar.search_events.CalendarSearchEvents.html)
- [CalendarUpdateEvent](https://python.langchain.com/api_reference/google_community/calendar/langchain_google_community.calendar.update_event.CalendarUpdateEvent.html)
- [GetCalendarsInfo](https://python.langchain.com/api_reference/google_community/calendar/langchain_google_community.calendar.get_calendars_info.GetCalendarsInfo.html)
- [CalendarMoveEvent](https://python.langchain.com/api_reference/google_community/calendar/langchain_google_community.calendar.move_event.CalendarMoveEvent.html)
- [CalendarDeleteEvent](https://python.langchain.com/api_reference/google_community/calendar/langchain_google_community.calendar.delete_event.CalendarDeleteEvent.html)
- [GetCurrentDatetime](https://python.langchain.com/api_reference/google_community/calendar/langchain_google_community.calendar.current_datetime.GetCurrentDatetime.html)

## 调用

### [直接使用参数调用](/docs/concepts/tools/#use-the-tool-directly)

你可以通过传递字典格式的必需参数来直接调用该工具。以下是使用 `CalendarCreateEvent` 工具创建新事件的示例。

In [ ]:
from langchain_google_community.calendar.create_event import CalendarCreateEvent

tool = CalendarCreateEvent()
tool.invoke(
    {
        "summary": "Calculus exam",
        "start_datetime": "2025-07-11 11:00:00",
        "end_datetime": "2025-07-11 13:00:00",
        "timezone": "America/Mexico_City",
        "location": "UAM Cuajimalpa",
        "description": "Event created from the LangChain toolkit",
        "reminders": [{"method": "popup", "minutes": 60}],
        "conference_data": True,
        "color_id": "5",
    }
)

'Event created: https://www.google.com/calendar/event?eid=amoxdjVsM2UzMW51Yjk2czc4ajhvaGdkcGcgam9yZ2VhbmczM0Bt'

## 在 Agent 中使用

下面我们将展示如何将该工具包整合到 [Agent](/docs/tutorials/agents) 中。

我们需要一个 LLM 或聊天模型：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [ ]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [10]:
example_query = "Create a green event for this afternoon to go for a 30-minute run."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Create a green event for this afternoon to go for a 30-minute run.
================================== Ai Message ==================================
Tool Calls:
  get_current_datetime (call_drHRRhm6pdvcAuqagONUEKs5)
 Call ID: call_drHRRhm6pdvcAuqagONUEKs5
  Args:
================================= Tool Message =================================
Name: get_current_datetime

Time zone: America/Mexico_City, Date and time: 2025-04-02 19:07:30
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_p60zSVMmmjTy5Ctezzmlb9zD)
 Call ID: call_p60zSVMmmjTy5Ctezzmlb9zD
  Args:
    summary: Run
    start_datetime: 2025-04-02 19:30:00
    end_datetime: 2025-04-02 20:00:00
    timezone: America/Mexico_City
    color_id: 2
================================= Tool Message =================================
Name: create_calendar_event

Event created: https://www.

## API 参考

- 请参阅 [Google Calendar API 概览](https://developers.google.com/calendar/api/guides/overview) 以获取 Google Calendar API 的更多详细信息。
- 有关所有 Google Calendar Toolkit 功能和配置的详细文档，请前往 [calendar 文档](https://python.langchain.com/api_reference/google_community/calendar.html)。